# Aerodynamic Surrogate Model Prototyping

## Overview
This notebook demonstrates the complete machine learning workflow for developing aerodynamic surrogate models using the Windsor body CFD dataset. We'll implement multiple regression algorithms, perform hyperparameter optimization, and evaluate models using both statistical and aerodynamic-specific metrics.

## Objectives
1. **Rapid Model Prototyping**: Quick evaluation of multiple algorithms
2. **Hyperparameter Optimization**: Find optimal configurations for best models
3. **Physics-Informed Validation**: Ensure models respect aerodynamic principles
4. **Feature Importance Analysis**: Understand which parameters drive predictions
5. **Model Comparison**: Comprehensive evaluation framework
6. **Production Readiness**: Prepare models for deployment

## Aerodynamic Context
- **Primary Targets**: Drag coefficient (Cd) and Lift coefficient (Cl)
- **Input Features**: 7 geometric parameters → engineered to 25+ features → optimally selected
- **Physical Validation**: Ensure predictions follow fluid mechanics principles
- **Performance Metrics**: R², RMSE, MAE, and aerodynamic-specific validation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys

# Add src directory to path for imports
sys.path.append('../src')

# Scientific computing
from scipy import stats
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Regression algorithms
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Project imports
from data_processing import (
    WindsorDataLoader, AerodynamicPreprocessor,
    create_drag_preprocessor, create_lift_preprocessor,
    create_multi_target_preprocessor
)
from train import AerodynamicModelTrainer

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Suppress warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")
print(f"Working directory: {Path.cwd()}")

## 1. Data Loading and Preprocessing

Load the Windsor body dataset and apply our advanced preprocessing pipeline.

In [ ]:
# Load the Windsor body dataset
print("Loading Windsor body dataset...")
loader = WindsorDataLoader()
features, targets = loader.get_feature_target_split()

print(f"Dataset loaded:")
print(f"  Features shape: {features.shape}")
print(f"  Targets shape: {targets.shape}")
print(f"  Feature columns: {features.columns.tolist()}")
print(f"  Target columns: {targets.columns.tolist()}")

# Display first few rows
print("\nFirst 5 rows of features:")
features.head()

In [ ]:
# Apply advanced preprocessing for drag coefficient prediction
print("Applying advanced preprocessing for drag coefficient prediction...")

# Create drag-specific preprocessor
drag_preprocessor = create_drag_preprocessor(
    n_features=12,
    feature_selection_method='combined',
    scaling_strategy='mixed'
)

# Prepare drag target
y_drag = targets[['cd']]

# Fit and transform
X_processed_drag, y_processed_drag = drag_preprocessor.fit_transform(features, y_drag)

print(f"\nDrag preprocessing results:")
print(f"  Original features: {features.shape[1]}")
print(f"  Processed features: {X_processed_drag.shape[1]}")
print(f"  Selected features: {X_processed_drag.columns.tolist() if hasattr(X_processed_drag, 'columns') else 'Array format'}")

# Display preprocessing report
print("\nPreprocessing Pipeline Report:")
print(drag_preprocessor.generate_preprocessing_report())

In [ ]:
# Create train-test split for drag prediction
X_train_drag, X_test_drag, y_train_drag, y_test_drag = drag_preprocessor.train_test_split(
    X_processed_drag, y_processed_drag
)

print(f"Train-test split for drag prediction:")
print(f"  Training set: {X_train_drag.shape}")
print(f"  Test set: {X_test_drag.shape}")
print(f"  Target statistics:")
print(f"    Train mean: {y_train_drag.mean().values[0]:.4f}")
print(f"    Train std: {y_train_drag.std().values[0]:.4f}")
print(f"    Test mean: {y_test_drag.mean().values[0]:.4f}")
print(f"    Test std: {y_test_drag.std().values[0]:.4f}")

## 2. Rapid Model Prototyping

Quickly evaluate multiple algorithms to identify the most promising approaches.

In [ ]:
# Define a set of algorithms for rapid prototyping
def rapid_model_evaluation(X_train, X_test, y_train, y_test, cv_folds=5):
    """
    Quickly evaluate multiple algorithms with default parameters.
    """
    # Convert to appropriate format
    y_train_flat = y_train.ravel() if y_train.ndim > 1 else y_train
    y_test_flat = y_test.ravel() if y_test.ndim > 1 else y_test
    
    # Define models for rapid evaluation
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge': Ridge(alpha=1.0),
        'Lasso': Lasso(alpha=0.1),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'SVR (RBF)': SVR(kernel='rbf', C=1.0),
        'MLP': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
    }
    
    results = []
    
    for name, model in models.items():
        print(f"Evaluating {name}...")
        
        try:
            # Cross-validation
            cv_scores = cross_val_score(model, X_train, y_train_flat, 
                                      cv=cv_folds, scoring='neg_mean_squared_error')
            cv_rmse = np.sqrt(-cv_scores)
            
            # Fit and predict
            model.fit(X_train, y_train_flat)
            y_pred = model.predict(X_test)
            
            # Calculate metrics
            r2 = r2_score(y_test_flat, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test_flat, y_pred))
            mae = mean_absolute_error(y_test_flat, y_pred)
            
            results.append({
                'Model': name,
                'CV_RMSE_Mean': cv_rmse.mean(),
                'CV_RMSE_Std': cv_rmse.std(),
                'Test_R2': r2,
                'Test_RMSE': rmse,
                'Test_MAE': mae,
                'Model_Object': model
            })
            
        except Exception as e:
            print(f"  Error with {name}: {str(e)}")
            continue
    
    return pd.DataFrame(results).sort_values('Test_R2', ascending=False)

# Run rapid evaluation
print("Starting rapid model evaluation for drag prediction...")
rapid_results = rapid_model_evaluation(X_train_drag, X_test_drag, y_train_drag, y_test_drag)

print("\n" + "="*60)
print("RAPID MODEL EVALUATION RESULTS")
print("="*60)
print(rapid_results[['Model', 'Test_R2', 'Test_RMSE', 'CV_RMSE_Mean']].round(4))

In [ ]:
# Visualize rapid evaluation results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# R² comparison
ax1 = axes[0]
models = rapid_results['Model'].values
r2_scores = rapid_results['Test_R2'].values
bars1 = ax1.bar(models, r2_scores, color='skyblue', alpha=0.8)
ax1.set_ylabel('R² Score')
ax1.set_title('Model Comparison - R² Score')
ax1.set_ylim(0, 1)
ax1.tick_params(axis='x', rotation=45)

# Add value labels
for bar, score in zip(bars1, r2_scores):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{score:.3f}', ha='center', va='bottom', fontsize=9)

# RMSE comparison
ax2 = axes[1]
rmse_scores = rapid_results['Test_RMSE'].values
bars2 = ax2.bar(models, rmse_scores, color='lightcoral', alpha=0.8)
ax2.set_ylabel('RMSE')
ax2.set_title('Model Comparison - RMSE')
ax2.tick_params(axis='x', rotation=45)

# Add value labels
for bar, score in zip(bars2, rmse_scores):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.suptitle('Rapid Model Evaluation - Drag Coefficient Prediction', y=1.02, fontsize=14)
plt.show()

# Identify top 3 models
top_models = rapid_results.head(3)
print(f"\nTop 3 performing models:")
for i, (_, row) in enumerate(top_models.iterrows(), 1):
    print(f"  {i}. {row['Model']:15s}: R² = {row['Test_R2']:.4f}, RMSE = {row['Test_RMSE']:.4f}")

## 3. Hyperparameter Optimization

Optimize hyperparameters for the top-performing models.

In [ ]:
# Hyperparameter optimization for Random Forest (typically a top performer)
print("Optimizing Random Forest hyperparameters...")

# Define parameter grid
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Random search for efficiency
rf_model = RandomForestRegressor(random_state=42)
rf_search = RandomizedSearchCV(
    rf_model, rf_param_grid, n_iter=30, cv=5, 
    scoring='neg_mean_squared_error', n_jobs=-1, random_state=42
)

# Fit and get results
y_train_flat = y_train_drag.ravel()
rf_search.fit(X_train_drag, y_train_flat)

print(f"Best Random Forest parameters:")
for param, value in rf_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV score (RMSE): {np.sqrt(-rf_search.best_score_):.4f}")

# Evaluate optimized model
best_rf = rf_search.best_estimator_
y_pred_rf = best_rf.predict(X_test_drag)
y_test_flat = y_test_drag.ravel()

rf_r2 = r2_score(y_test_flat, y_pred_rf)
rf_rmse = np.sqrt(mean_squared_error(y_test_flat, y_pred_rf))

print(f"Optimized Random Forest performance:")
print(f"  Test R²: {rf_r2:.4f}")
print(f"  Test RMSE: {rf_rmse:.4f}")

In [ ]:
# Hyperparameter optimization for Gradient Boosting
print("Optimizing Gradient Boosting hyperparameters...")

# Define parameter grid
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'min_samples_split': [2, 5, 10]
}

# Random search
gb_model = GradientBoostingRegressor(random_state=42)
gb_search = RandomizedSearchCV(
    gb_model, gb_param_grid, n_iter=25, cv=5,
    scoring='neg_mean_squared_error', n_jobs=-1, random_state=42
)

gb_search.fit(X_train_drag, y_train_flat)

print(f"Best Gradient Boosting parameters:")
for param, value in gb_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV score (RMSE): {np.sqrt(-gb_search.best_score_):.4f}")

# Evaluate optimized model
best_gb = gb_search.best_estimator_
y_pred_gb = best_gb.predict(X_test_drag)

gb_r2 = r2_score(y_test_flat, y_pred_gb)
gb_rmse = np.sqrt(mean_squared_error(y_test_flat, y_pred_gb))

print(f"Optimized Gradient Boosting performance:")
print(f"  Test R²: {gb_r2:.4f}")
print(f"  Test RMSE: {gb_rmse:.4f}")

In [ ]:
# Compare optimized models
optimized_results = pd.DataFrame([
    {'Model': 'Random Forest (Optimized)', 'R2': rf_r2, 'RMSE': rf_rmse},
    {'Model': 'Gradient Boosting (Optimized)', 'R2': gb_r2, 'RMSE': gb_rmse}
])

print("\n" + "="*50)
print("OPTIMIZED MODEL COMPARISON")
print("="*50)
print(optimized_results.round(4))

# Determine best model
best_model_idx = optimized_results['R2'].idxmax()
best_model_name = optimized_results.loc[best_model_idx, 'Model']
best_model = best_rf if 'Random Forest' in best_model_name else best_gb
best_predictions = y_pred_rf if 'Random Forest' in best_model_name else y_pred_gb

print(f"\nBest performing model: {best_model_name}")
print(f"Best R²: {optimized_results.loc[best_model_idx, 'R2']:.4f}")
print(f"Best RMSE: {optimized_results.loc[best_model_idx, 'RMSE']:.4f}")

## 4. Model Evaluation and Visualization

Comprehensive evaluation including prediction plots and residual analysis.

In [ ]:
# Create comprehensive evaluation plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Prediction vs Actual
ax1 = axes[0, 0]
ax1.scatter(y_test_flat, best_predictions, alpha=0.6, s=50)
ax1.plot([y_test_flat.min(), y_test_flat.max()], [y_test_flat.min(), y_test_flat.max()], 'r--', lw=2)
ax1.set_xlabel('True Drag Coefficient (Cd)')
ax1.set_ylabel('Predicted Drag Coefficient (Cd)')
ax1.set_title(f'{best_model_name}\nPrediction vs Actual\nR² = {optimized_results.loc[best_model_idx, "R2"]:.4f}')
ax1.grid(True, alpha=0.3)

# Add confidence bounds
perfect_line = np.linspace(y_test_flat.min(), y_test_flat.max(), 100)
error_margin = 0.05  # 5% error margin
ax1.fill_between(perfect_line, perfect_line*(1-error_margin), perfect_line*(1+error_margin), 
                alpha=0.2, color='gray', label='±5% Error Band')
ax1.legend()

# Plot 2: Residuals
ax2 = axes[0, 1]
residuals = best_predictions - y_test_flat
ax2.scatter(best_predictions, residuals, alpha=0.6, s=50)
ax2.axhline(y=0, color='r', linestyle='--')
ax2.set_xlabel('Predicted Drag Coefficient (Cd)')
ax2.set_ylabel('Residuals')
ax2.set_title('Residual Analysis')
ax2.grid(True, alpha=0.3)

# Add statistical info
mean_residual = np.mean(residuals)
std_residual = np.std(residuals)
ax2.text(0.05, 0.95, f'Mean: {mean_residual:.4f}\nStd: {std_residual:.4f}', 
         transform=ax2.transAxes, verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Plot 3: Error Distribution
ax3 = axes[1, 0]
ax3.hist(residuals, bins=20, alpha=0.7, density=True, edgecolor='black')
ax3.axvline(0, color='r', linestyle='--', label='Zero Error')
ax3.set_xlabel('Residuals')
ax3.set_ylabel('Density')
ax3.set_title('Residual Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Overlay normal distribution
x_norm = np.linspace(residuals.min(), residuals.max(), 100)
normal_dist = stats.norm.pdf(x_norm, mean_residual, std_residual)
ax3.plot(x_norm, normal_dist, 'r-', linewidth=2, label='Normal Fit')
ax3.legend()

# Plot 4: Learning Curve (if tree-based model)
ax4 = axes[1, 1]
if hasattr(best_model, 'estimators_'):
    # For ensemble methods, show feature importance
    feature_names = X_train_drag.columns if hasattr(X_train_drag, 'columns') else [f'Feature_{i}' for i in range(X_train_drag.shape[1])]
    importance = best_model.feature_importances_
    
    # Sort by importance
    sorted_idx = np.argsort(importance)[-10:]  # Top 10
    
    ax4.barh(range(len(sorted_idx)), importance[sorted_idx])
    ax4.set_yticks(range(len(sorted_idx)))
    ax4.set_yticklabels([feature_names[i] for i in sorted_idx])
    ax4.set_xlabel('Feature Importance')
    ax4.set_title('Top 10 Feature Importance')
    ax4.grid(True, alpha=0.3)
else:
    # For other models, show Q-Q plot
    from scipy.stats import probplot
    probplot(residuals, dist="norm", plot=ax4)
    ax4.set_title('Q-Q Plot (Normality Test)')
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Comprehensive Model Evaluation - Drag Coefficient Prediction', y=1.02, fontsize=16)
plt.show()

## 5. Physics-Informed Validation

Validate that model predictions follow aerodynamic principles.

In [ ]:
# Physics validation for drag predictions
def validate_aerodynamic_physics(model, X_test, feature_names):
    """
    Validate model predictions against known aerodynamic principles.
    """
    print("Validating aerodynamic physics...")
    
    # Get predictions
    predictions = model.predict(X_test)
    
    # Convert to DataFrame for easier manipulation
    if hasattr(X_test, 'columns'):
        X_df = X_test
    else:
        X_df = pd.DataFrame(X_test, columns=feature_names)
    
    validation_results = {}
    
    # Test 1: Frontal area vs drag (should be positive correlation)
    if 'frontal_area' in X_df.columns:
        frontal_area_corr = np.corrcoef(X_df['frontal_area'], predictions)[0, 1]
        validation_results['frontal_area_drag'] = {
            'correlation': frontal_area_corr,
            'expected': 'positive',
            'is_correct': frontal_area_corr > 0,
            'interpretation': 'Larger frontal area should increase drag due to higher blockage ratio'
        }
    
    # Test 2: Clearance effects (complex relationship)
    if 'clearance' in X_df.columns:
        clearance_corr = np.corrcoef(X_df['clearance'], predictions)[0, 1]
        validation_results['clearance_drag'] = {
            'correlation': clearance_corr,
            'expected': 'complex (can be positive or negative)',
            'is_reasonable': abs(clearance_corr) < 0.8,  # Should not be extremely strong
            'interpretation': 'Clearance affects underbody flow and ground effect'
        }
    
    # Test 3: Physical bounds check
    drag_min, drag_max = predictions.min(), predictions.max()
    expected_min, expected_max = 0.1, 1.0  # Reasonable automotive drag range
    
    validation_results['physical_bounds'] = {
        'min_predicted': drag_min,
        'max_predicted': drag_max,
        'expected_range': (expected_min, expected_max),
        'within_bounds': (drag_min >= expected_min * 0.8) and (drag_max <= expected_max * 1.2),
        'interpretation': 'Drag coefficients should be within reasonable automotive range'
    }
    
    # Test 4: Prediction variance (should not be too extreme)
    pred_std = np.std(predictions)
    pred_mean = np.mean(predictions)
    coeff_variation = pred_std / pred_mean
    
    validation_results['prediction_variance'] = {
        'coefficient_of_variation': coeff_variation,
        'is_reasonable': coeff_variation < 0.5,  # Should not vary more than 50%
        'interpretation': 'Predictions should show reasonable variance across design space'
    }
    
    return validation_results

# Run physics validation
feature_names = X_train_drag.columns if hasattr(X_train_drag, 'columns') else [f'feature_{i}' for i in range(X_train_drag.shape[1])]
physics_results = validate_aerodynamic_physics(best_model, X_test_drag, feature_names)

print("\n" + "="*60)
print("PHYSICS VALIDATION RESULTS")
print("="*60)

for test_name, results in physics_results.items():
    print(f"\n{test_name.upper().replace('_', ' ')}:")
    for key, value in results.items():
        if key == 'interpretation':
            print(f"  💡 {value}")
        elif key in ['is_correct', 'is_reasonable', 'within_bounds']:
            status = "✅ PASS" if value else "⚠️ WARNING"
            print(f"  {status}")
        else:
            print(f"  {key}: {value}")

## 6. Multi-Target Model Training

Train models to predict both drag and lift coefficients simultaneously.

In [ ]:
# Multi-target preprocessing for both drag and lift
print("Preparing multi-target model for drag and lift prediction...")

# Create multi-target preprocessor
multi_preprocessor = create_multi_target_preprocessor(
    n_features=15,
    feature_selection_method='combined',
    scaling_strategy='mixed'
)

# Use both drag and lift targets
y_multi = targets[['cd', 'cl']]

# Fit and transform
X_processed_multi, y_processed_multi = multi_preprocessor.fit_transform(features, y_multi)

# Create train-test split
X_train_multi, X_test_multi, y_train_multi, y_test_multi = multi_preprocessor.train_test_split(
    X_processed_multi, y_processed_multi
)

print(f"Multi-target dataset:")
print(f"  Training features: {X_train_multi.shape}")
print(f"  Training targets: {y_train_multi.shape}")
print(f"  Selected features: {X_processed_multi.shape[1]}")
print(f"  Target columns: {y_multi.columns.tolist()}")

In [ ]:
# Train multi-target models
from sklearn.multioutput import MultiOutputRegressor

print("Training multi-target models...")

# Define multi-target models
multi_models = {
    'Multi-RF': MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)),
    'Multi-GB': MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=42)),
    'Multi-SVR': MultiOutputRegressor(SVR(kernel='rbf'))
}

multi_results = []

for name, model in multi_models.items():
    print(f"  Training {name}...")
    
    # Fit model
    model.fit(X_train_multi, y_train_multi)
    
    # Predict
    y_pred_multi = model.predict(X_test_multi)
    
    # Calculate metrics for each target
    cd_r2 = r2_score(y_test_multi.iloc[:, 0], y_pred_multi[:, 0])
    cd_rmse = np.sqrt(mean_squared_error(y_test_multi.iloc[:, 0], y_pred_multi[:, 0]))
    
    cl_r2 = r2_score(y_test_multi.iloc[:, 1], y_pred_multi[:, 1])
    cl_rmse = np.sqrt(mean_squared_error(y_test_multi.iloc[:, 1], y_pred_multi[:, 1]))
    
    # Overall metrics
    overall_r2 = (cd_r2 + cl_r2) / 2
    overall_rmse = (cd_rmse + cl_rmse) / 2
    
    multi_results.append({
        'Model': name,
        'Cd_R2': cd_r2,
        'Cd_RMSE': cd_rmse,
        'Cl_R2': cl_r2,
        'Cl_RMSE': cl_rmse,
        'Overall_R2': overall_r2,
        'Overall_RMSE': overall_rmse,
        'Model_Object': model,
        'Predictions': y_pred_multi
    })

multi_df = pd.DataFrame(multi_results).sort_values('Overall_R2', ascending=False)

print("\n" + "="*70)
print("MULTI-TARGET MODEL RESULTS")
print("="*70)
print(multi_df[['Model', 'Cd_R2', 'Cd_RMSE', 'Cl_R2', 'Cl_RMSE', 'Overall_R2']].round(4))

In [ ]:
# Visualize multi-target results
best_multi_model = multi_df.iloc[0]['Model_Object']
best_multi_pred = multi_df.iloc[0]['Predictions']
best_multi_name = multi_df.iloc[0]['Model']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Drag coefficient predictions
ax1 = axes[0, 0]
cd_true = y_test_multi.iloc[:, 0]
cd_pred = best_multi_pred[:, 0]
ax1.scatter(cd_true, cd_pred, alpha=0.6, s=50, color='blue')
ax1.plot([cd_true.min(), cd_true.max()], [cd_true.min(), cd_true.max()], 'r--', lw=2)
ax1.set_xlabel('True Drag Coefficient (Cd)')
ax1.set_ylabel('Predicted Drag Coefficient (Cd)')
ax1.set_title(f'{best_multi_name} - Drag Prediction\nR² = {multi_df.iloc[0]["Cd_R2"]:.4f}')
ax1.grid(True, alpha=0.3)

# Lift coefficient predictions
ax2 = axes[0, 1]
cl_true = y_test_multi.iloc[:, 1]
cl_pred = best_multi_pred[:, 1]
ax2.scatter(cl_true, cl_pred, alpha=0.6, s=50, color='green')
ax2.plot([cl_true.min(), cl_true.max()], [cl_true.min(), cl_true.max()], 'r--', lw=2)
ax2.set_xlabel('True Lift Coefficient (Cl)')
ax2.set_ylabel('Predicted Lift Coefficient (Cl)')
ax2.set_title(f'{best_multi_name} - Lift Prediction\nR² = {multi_df.iloc[0]["Cl_R2"]:.4f}')
ax2.grid(True, alpha=0.3)

# Residuals for drag
ax3 = axes[1, 0]
cd_residuals = cd_pred - cd_true
ax3.scatter(cd_pred, cd_residuals, alpha=0.6, s=50, color='blue')
ax3.axhline(y=0, color='r', linestyle='--')
ax3.set_xlabel('Predicted Drag Coefficient (Cd)')
ax3.set_ylabel('Residuals')
ax3.set_title('Drag Coefficient Residuals')
ax3.grid(True, alpha=0.3)

# Residuals for lift
ax4 = axes[1, 1]
cl_residuals = cl_pred - cl_true
ax4.scatter(cl_pred, cl_residuals, alpha=0.6, s=50, color='green')
ax4.axhline(y=0, color='r', linestyle='--')
ax4.set_xlabel('Predicted Lift Coefficient (Cl)')
ax4.set_ylabel('Residuals')
ax4.set_title('Lift Coefficient Residuals')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Multi-Target Model Evaluation - Drag and Lift Prediction', y=1.02, fontsize=16)
plt.show()

# Performance summary
print(f"\nBest multi-target model: {best_multi_name}")
print(f"Drag coefficient R²: {multi_df.iloc[0]['Cd_R2']:.4f}")
print(f"Lift coefficient R²: {multi_df.iloc[0]['Cl_R2']:.4f}")
print(f"Overall R²: {multi_df.iloc[0]['Overall_R2']:.4f}")

## 7. Feature Importance and Interpretability

Analyze which features are most important for aerodynamic predictions.

In [ ]:
# Feature importance analysis
def analyze_feature_importance(model, feature_names, model_name):
    """
    Extract and analyze feature importance from trained models.
    """
    importance_data = []
    
    if hasattr(model, 'feature_importances_'):
        # Tree-based models
        importance = model.feature_importances_
        importance_data = list(zip(feature_names, importance))
        
    elif hasattr(model, 'estimators_') and hasattr(model.estimators_[0], 'feature_importances_'):
        # MultiOutput with tree-based estimators
        # Average importance across all estimators
        all_importance = np.array([est.feature_importances_ for est in model.estimators_])
        avg_importance = np.mean(all_importance, axis=0)
        importance_data = list(zip(feature_names, avg_importance))
        
    elif hasattr(model, 'coef_'):
        # Linear models
        coef = model.coef_
        if coef.ndim > 1:
            # Multi-output: average absolute coefficients
            importance = np.mean(np.abs(coef), axis=0)
        else:
            importance = np.abs(coef)
        importance_data = list(zip(feature_names, importance))
    
    if importance_data:
        # Sort by importance
        importance_data.sort(key=lambda x: x[1], reverse=True)
        
        # Create DataFrame
        importance_df = pd.DataFrame(importance_data, columns=['Feature', 'Importance'])
        importance_df['Normalized_Importance'] = importance_df['Importance'] / importance_df['Importance'].sum()
        
        return importance_df
    
    return None

# Analyze feature importance for best single-target model
feature_names_drag = X_train_drag.columns if hasattr(X_train_drag, 'columns') else [f'feature_{i}' for i in range(X_train_drag.shape[1])]
importance_drag = analyze_feature_importance(best_model, feature_names_drag, best_model_name)

if importance_drag is not None:
    print("\n" + "="*60)
    print(f"FEATURE IMPORTANCE - {best_model_name.upper()}")
    print("="*60)
    print(importance_drag.head(10).round(4))
    
    # Visualize feature importance
    plt.figure(figsize=(12, 8))
    
    # Top 10 features
    top_features = importance_drag.head(10)
    
    plt.barh(range(len(top_features)), top_features['Importance'], color='skyblue')
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 10 Feature Importance - {best_model_name}\nDrag Coefficient Prediction')
    plt.gca().invert_yaxis()
    
    # Add value labels
    for i, (feature, importance) in enumerate(zip(top_features['Feature'], top_features['Importance'])):
        plt.text(importance + 0.001, i, f'{importance:.3f}', va='center', fontsize=9)
    
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
else:
    print(f"Feature importance not available for {best_model_name}")

In [ ]:
# Analyze feature importance for multi-target model
feature_names_multi = X_train_multi.columns if hasattr(X_train_multi, 'columns') else [f'feature_{i}' for i in range(X_train_multi.shape[1])]
importance_multi = analyze_feature_importance(best_multi_model, feature_names_multi, best_multi_name)

if importance_multi is not None:
    print("\n" + "="*60)
    print(f"FEATURE IMPORTANCE - {best_multi_name.upper()} (MULTI-TARGET)")
    print("="*60)
    print(importance_multi.head(10).round(4))
    
    # Compare feature importance between models
    if importance_drag is not None:
        plt.figure(figsize=(14, 10))
        
        # Get common features
        common_features = set(importance_drag['Feature']).intersection(set(importance_multi['Feature']))
        
        if common_features:
            # Create comparison DataFrame
            comparison_data = []
            for feature in common_features:
                drag_imp = importance_drag[importance_drag['Feature'] == feature]['Normalized_Importance'].values[0]
                multi_imp = importance_multi[importance_multi['Feature'] == feature]['Normalized_Importance'].values[0]
                comparison_data.append({
                    'Feature': feature,
                    'Drag_Model': drag_imp,
                    'Multi_Model': multi_imp
                })
            
            comparison_df = pd.DataFrame(comparison_data)
            comparison_df['Avg_Importance'] = (comparison_df['Drag_Model'] + comparison_df['Multi_Model']) / 2
            comparison_df = comparison_df.sort_values('Avg_Importance', ascending=False).head(10)
            
            # Create comparison plot
            x = np.arange(len(comparison_df))
            width = 0.35
            
            plt.bar(x - width/2, comparison_df['Drag_Model'], width, label='Drag-Only Model', alpha=0.8)
            plt.bar(x + width/2, comparison_df['Multi_Model'], width, label='Multi-Target Model', alpha=0.8)
            
            plt.xlabel('Features')
            plt.ylabel('Normalized Feature Importance')
            plt.title('Feature Importance Comparison\nDrag-Only vs Multi-Target Models')
            plt.xticks(x, comparison_df['Feature'], rotation=45, ha='right')
            plt.legend()
            plt.grid(True, alpha=0.3, axis='y')
            plt.tight_layout()
            plt.show()
            
            print("\nFeature Importance Comparison (Top 10):")
            print(comparison_df[['Feature', 'Drag_Model', 'Multi_Model']].round(4))
        else:
            print("No common features found between models for comparison.")
else:
    print(f"Feature importance not available for {best_multi_name}")

## 8. Model Persistence and Production Readiness

Save the best models and create a simple prediction interface.

In [ ]:
# Save the best models and preprocessors
import pickle
from datetime import datetime

# Create models directory if it doesn't exist
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save drag model and preprocessor
drag_model_path = models_dir / f'best_drag_model_{timestamp}.pkl'
drag_preprocessor_path = models_dir / f'drag_preprocessor_{timestamp}.pkl'

with open(drag_model_path, 'wb') as f:
    pickle.dump(best_model, f)

with open(drag_preprocessor_path, 'wb') as f:
    pickle.dump(drag_preprocessor, f)

print(f"Drag model saved to: {drag_model_path}")
print(f"Drag preprocessor saved to: {drag_preprocessor_path}")

# Save multi-target model and preprocessor
multi_model_path = models_dir / f'best_multi_model_{timestamp}.pkl'
multi_preprocessor_path = models_dir / f'multi_preprocessor_{timestamp}.pkl'

with open(multi_model_path, 'wb') as f:
    pickle.dump(best_multi_model, f)

with open(multi_preprocessor_path, 'wb') as f:
    pickle.dump(multi_preprocessor, f)

print(f"Multi-target model saved to: {multi_model_path}")
print(f"Multi-target preprocessor saved to: {multi_preprocessor_path}")

# Save model metadata
metadata = {
    'timestamp': timestamp,
    'best_drag_model': {
        'name': best_model_name,
        'r2_score': optimized_results.loc[best_model_idx, 'R2'],
        'rmse': optimized_results.loc[best_model_idx, 'RMSE'],
        'model_path': str(drag_model_path),
        'preprocessor_path': str(drag_preprocessor_path)
    },
    'best_multi_model': {
        'name': best_multi_name,
        'cd_r2': multi_df.iloc[0]['Cd_R2'],
        'cl_r2': multi_df.iloc[0]['Cl_R2'],
        'overall_r2': multi_df.iloc[0]['Overall_R2'],
        'model_path': str(multi_model_path),
        'preprocessor_path': str(multi_preprocessor_path)
    },
    'dataset_info': {
        'original_features': features.shape[1],
        'processed_features_drag': X_processed_drag.shape[1],
        'processed_features_multi': X_processed_multi.shape[1],
        'samples': features.shape[0]
    }
}

metadata_path = models_dir / f'model_metadata_{timestamp}.json'
import json
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"Model metadata saved to: {metadata_path}")

In [ ]:
# Create a simple prediction interface
def create_prediction_interface():
    """
    Create a simple interface for making predictions with trained models.
    """
    class AerodynamicPredictor:
        def __init__(self, drag_model, drag_preprocessor, multi_model, multi_preprocessor):
            self.drag_model = drag_model
            self.drag_preprocessor = drag_preprocessor
            self.multi_model = multi_model
            self.multi_preprocessor = multi_preprocessor
        
        def predict_drag(self, geometric_params):
            """
            Predict drag coefficient from geometric parameters.
            
            Args:
                geometric_params: DataFrame or dict with geometric parameters
            
            Returns:
                Predicted drag coefficient
            """
            if isinstance(geometric_params, dict):
                geometric_params = pd.DataFrame([geometric_params])
            
            # Preprocess
            X_processed = self.drag_preprocessor.transform(geometric_params)
            
            # Predict
            prediction = self.drag_model.predict(X_processed)
            
            return prediction[0] if len(prediction) == 1 else prediction
        
        def predict_multi(self, geometric_params):
            """
            Predict both drag and lift coefficients from geometric parameters.
            
            Args:
                geometric_params: DataFrame or dict with geometric parameters
            
            Returns:
                Dictionary with 'cd' and 'cl' predictions
            """
            if isinstance(geometric_params, dict):
                geometric_params = pd.DataFrame([geometric_params])
            
            # Preprocess
            X_processed = self.multi_preprocessor.transform(geometric_params)
            
            # Predict
            predictions = self.multi_model.predict(X_processed)
            
            if predictions.ndim == 1:
                return {'cd': predictions[0], 'cl': predictions[1]}
            else:
                return [{'cd': pred[0], 'cl': pred[1]} for pred in predictions]
        
        def validate_inputs(self, geometric_params):
            """
            Validate input parameters are within reasonable ranges.
            """
            required_params = [
                'ratio_length_back_fast',
                'ratio_height_nose_windshield', 
                'ratio_height_fast_back',
                'side_taper',
                'clearance',
                'bottom_taper_angle',
                'frontal_area'
            ]
            
            if isinstance(geometric_params, dict):
                missing = set(required_params) - set(geometric_params.keys())
                if missing:
                    return False, f"Missing parameters: {missing}"
            
            return True, "Valid inputs"
    
    return AerodynamicPredictor(best_model, drag_preprocessor, best_multi_model, multi_preprocessor)

# Create predictor instance
predictor = create_prediction_interface()

print("Prediction interface created successfully!")
print("\nExample usage:")
print("predictor.predict_drag(geometric_parameters)")
print("predictor.predict_multi(geometric_parameters)")

In [ ]:
# Demonstrate the prediction interface with a test case
print("Testing prediction interface with sample geometric parameters...")

# Create a test case with reasonable geometric parameters
test_geometry = {
    'ratio_length_back_fast': 0.5,
    'ratio_height_nose_windshield': 0.3,
    'ratio_height_fast_back': 0.2,
    'side_taper': 15.0,
    'clearance': 100.0,
    'bottom_taper_angle': 10.0,
    'frontal_area': 0.08
}

# Validate inputs
is_valid, message = predictor.validate_inputs(test_geometry)
print(f"Input validation: {message}")

if is_valid:
    # Test drag prediction
    drag_pred = predictor.predict_drag(test_geometry)
    print(f"\nDrag coefficient prediction: {drag_pred:.4f}")
    
    # Test multi-target prediction
    multi_pred = predictor.predict_multi(test_geometry)
    print(f"Multi-target predictions:")
    print(f"  Drag coefficient (Cd): {multi_pred['cd']:.4f}")
    print(f"  Lift coefficient (Cl): {multi_pred['cl']:.4f}")
    
    # Physical interpretation
    print(f"\nPhysical interpretation:")
    if multi_pred['cd'] < 0.3:
        print(f"  🚗 Low drag configuration - excellent for fuel efficiency")
    elif multi_pred['cd'] < 0.4:
        print(f"  🚙 Moderate drag - typical automotive range")
    else:
        print(f"  🚛 High drag - may need aerodynamic optimization")
    
    if multi_pred['cl'] < -0.1:
        print(f"  ⬇️ Generates downforce - improves high-speed stability")
    elif multi_pred['cl'] > 0.1:
        print(f"  ⬆️ Generates lift - may reduce traction at high speeds")
    else:
        print(f"  ➡️ Neutral lift - balanced aerodynamic behavior")

print("\n" + "="*60)
print("MODEL PROTOTYPING COMPLETE")
print("="*60)
print(f"✅ Successfully trained and validated aerodynamic surrogate models")
print(f"✅ Best drag model: {best_model_name} (R² = {optimized_results.loc[best_model_idx, 'R2']:.4f})")
print(f"✅ Best multi-target model: {best_multi_name} (Overall R² = {multi_df.iloc[0]['Overall_R2']:.4f})")
print(f"✅ Models validated against aerodynamic physics principles")
print(f"✅ Feature importance analysis completed")
print(f"✅ Models saved and ready for production deployment")
print(f"✅ Prediction interface created and tested")